# Most frequent builds

See what builds are most commonly invoked by developers, e.g. `clean assemble`, `test` or `check`. You can [set up the URL and a token for your Develocity instance](https://github.com/gabrielfeo/develocity-api-kotlin/blob/main/README.md#setup) and run this notebook as-is for your own project.

This is a simple example of something you can do with the API. It could bring insights, for example:

- "Our developers frequently `clean` together with `assemble`. We should ask them why, because they shouldn't have to. Just an old habit from Maven or are they working around a build issue we don't know about?"
- "Some are doing `check` builds locally, which we set up to trigger our notably slow legacy tests. We should suggest they run `test` instead, leaving `check` for CI to run."

This notebook will take you through using develocity-api-kotlin in Jupyter, but it won't get into what a notebook is and how to run it. If you're not familiar with Jupyter:

- [Kotlin for data science overview](https://kotlinlang.org/docs/data-science-overview.html)
- [Kotlin for Jupyter notebooks](https://github.com/cheptsov/kotlin-jupyter-demo/blob/master/index.ipynb)

Note: GitHub preview won't render tables or graphs. I recommend previewing this in the [online Jupyter nbviewer](https://nbviewer.org/github/gabrielfeo/develocity-api-kotlin/blob/main/examples/example-notebooks/MostFrequentBuilds.ipynb).

## Setup

Add libraries to use, via line magics. `%use` is a [line magic](https://github.com/Kotlin/kotlin-jupyter#line-magics) of the Kotlin kernel that can do much more than adding the library. To illustrate, this setup can be replaced with a single line magic.

```kotlin
@file:DependsOn("com.gabrielfeo:develocity-api-kotlin:2023.4.0")

import com.gabrielfeo.gradle.enterprise.api.*
import com.gabrielfeo.gradle.enterprise.api.model.*
import com.gabrielfeo.gradle.enterprise.api.extension.*
```

is the same as:

```
%use develocity-api-kotlin(version=2023.4.0)

```

In [1]:
%useLatestDescriptors
%use develocitytlin(version=2023.4.0)
%use coroutines(v=1.7.1)

val api = DevelocityApi.newInstance()

## Fetch builds

Use [getBuildsFlow][1] to fetch all builds for a [query][2] with `/api/builds`.

By default, "builds" from the API are just an ID and an upload time, but we can request more info to come in the same 
response using the `models` parameter. "Models" are build details that would come from other endpoints. For example, 
requesting models=[gradleAttributes][3] brings data from `/api/builds/{id}/gradle-attributes` in the same response.

[1]: https://gabrielfeo.github.io/develocity-api-kotlin/library/com.gabrielfeo.gradle.enterprise.api.extension/get-builds-flow.html
[2]: https://docs.gradle.com/enterprise/api-manual/#advanced_search_syntax 
[3]: https://gabrielfeo.github.io/develocity-api-kotlin/library/com.gabrielfeo.gradle.enterprise.api
.model/-build-model-name/gradle-attributes/index.html

In [2]:
import java.time.temporal.*
import java.util.LinkedList

val builds: List<GradleAttributes> = runBlocking {
    api.buildsApi.getBuildsFlow(
        fromInstant = 0,
        query = """buildStartTime<-7d tag:local buildOutcome:failed""",
        models = listOf(BuildModelName.gradleAttributes),
    ).map {
        it.models!!.gradleAttributes!!.model!!
    }.toList(LinkedList())
}

println("${builds.size} builds")

17907 builds


## Tables

We'll now use [Kotlin/dataframe](https://github.com/Kotlin/dataframe) to visualize data

In [3]:
%use dataframe(v=0.13.1)

Use `List.toDataFrame` to create a table of builds

In [4]:
val buildCounts = builds.toDataFrame {
    "tasks" from { build ->
        val tasks = build.requestedTasks.joinToString(" ").trim(':')
        if (tasks.isNotBlank()) tasks
        else "IDE sync"
    }
}.groupBy("tasks").aggregate {
    count() into "count"
}.sortByDesc("count")

// Jupyter will render the last cell line (a String, an Int, a DataFrame, etc.)
buildCounts

tasks,count
app:assembleBrazilDebug,4677
kotlinLSPProjectDeps,577
IDE sync,420
feature:home:impl:testReleaseUnitTest...,112
feature:hits:cards:impl:testReleaseUn...,103
feature:splash:impl:testReleaseUnitTe...,96
feature:checkout:core:impl:testReleas...,92
feature:chat:core:impl:testReleaseUni...,90
feature:chat:core:impl:testReleaseUni...,73
feature:checkout:core:impl:testReleas...,70


## Plotting

We'll use [Kotlin/kandy](https://github.com/Kotlin/kandy) for plotting the table. We'll only plot the top 5.

In [5]:
%use kandy(v=0.6.0)

In [13]:
plot(buildCounts.take(3)) {
    barsH {
        x("count")
        y("tasks")
    }
    layout.size = 800 to 250
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="uvmcNH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"count":[4677.0,577.0,420.0],
"tasks":["app:assembleBrazilDebug","kotlinLSPProjectDeps","IDE sync"]
},
"ggsize":{
"width":800.0,
"height":250.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"count",
"y":"tasks"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("uvmcNH");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 3,500 
 
 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 
 
 4,500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 app:assembleBrazilDebug 
 
 
 
 
 
 
 kotlinLSPProjectDeps 
 
 
 
 
 
 
 IDE sync 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 tasks 
 
 
 
 
 count